# Vežba 12 : Linearna regresija - Metode semplovanja, selekcija parametara i regularizacija

**Za potrebe reprodukovanja analize, treba instalirati paket ISLP (koji su kreirali autori knjige) u okruženje bio_ml!** Uputstvo za instaliranje, kao i podaci koji se koriste u knjizi su na [pratećem sajtu knjige](https://www.statlearning.com/resources-python). Iskoristite sledeće komande:

`conda activate bio_ml`

`pip install ISLP`

`pip install l0bnb`

`conda list ISLP`

**Praktikum reprodukuje primer** koda dat u poglavlju 3.6 "Lab: Linear Regression" u ISLP knjizi, sa idejom da se on iskoristi kao templejt za pisanje sličnog koda za analizu drugog seta podataka.

## Set podataka 

**abalone_dataset.csv** sadrži tabelu u kojoj se za svaku od 4177 jedinki školjke Petrovo uho (eng. abalone, Haliotis sp.) nalaze vrednosti 9 veličina kojima odgovaraju sledeći nazivi kolona:
1. `Length` (Longest shell measurement) in mm
2. `Diameter` (perpendicular to length) in mm
3. `Height` (with meat in shell) in mm
4. `Whole weight` (whole abalone) in grams
5. `Shucked weight` (weight of meat) in grams
6. `Viscera weight`	(gut weight after bleeding) in grams
7. `Shell weight` (after being dried) in grams
8. `Rings` (+1.5 gives the age in years) as integer values (celi brojevi)
9. `Sex` as categorical with values M (male), F (female), and I (infant)

Starost ovih školjki se određuje presecanjem ljušture i brojanjem prstenova na obojenom preseku pod mikroskopom. Ova kolekcija podataka je sastavljena sa ciljem da se ispita da li se druge karakteristike školjke koje je lakše izmeriti mogu iskoristiti za predviđanje broja prstenova, tj. starosti školjke.

## Cilj vežbe

Dati kod u sekcijama zadataka je anotiran sa njegovom funkcijom.

Neophodno je izmeniti i dopuniti postojeći kod kako bi radio sa abalone_dataset.csv setom podataka koji je opisan iznad.




## Zadatak 1 - Pristup validacionog skupa, Polinomska regresija i Unakrsna validacija (LOOCV, kFold)
U ovom zadatku vršimo podelu seta podataka na test i validacioni set i računamo MSE (srednju kvadratnu grešku) na validacionom setu podataka.

Nakon toga radimo regresiju za više modela sa polinomskim članom različitog stepena.

Na kraju radimo unakrsnu validaciju (Cross-Validation).

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Ucitavanje podataka
abalone = pd.read_csv("abalone_dataset.csv")

# Nezavisna i zavisna promenljiva
X = abalone[["Shell weight"]].values
y = abalone["Rings"].values

# Podela na trening i validacioni skup
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=0
)

# Linearna regresija
model = LinearRegression()
model.fit(X_train, y_train)

# Predikcija i MSE
y_hat = model.predict(X_valid)
MSE = mean_squared_error(y_valid, y_hat)

print(f"MSE na validacionom skupu = {MSE:.2f}")

ModuleNotFoundError: No module named 'pandas'

# 1.1 Pristup validacionog skupa (The Validation Set Approach)
Neophodno je modifikovati set podataka na kome se radi. 

Zavisna varijabla (y_train, y_valid) treba da bude Rings kolona iz abalone_dataset.csv

Nezavisna varijabla treba da bude Shell weight kolona iz abalone_dataset.csv


In [ ]:
# Učitavanje 'Auto' skupa podataka
Auto = load_data('Auto')

# Podela podataka na trening i validacioni skup
# test_size=196 označava da 196 opservacija ide u validacioni skup
# random_state=0 osigurava ponovljivost rezultata pri svakom pokretanju
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

# Prilagođavanje modela linearne regresije koristeći samo trening skup
# Kreiranje specifikacije modela za 'horsepower' (konjske snage)
hp_mm = MS(['horsepower'])
# Transformacija trening podataka na osnovu specifikacije (dodaje presečnu tačku/intercept)
X_train = hp_mm.fit_transform(Auto_train)
# Definisanje zavisne varijable (odziva) za trening ('mpg' - milja po galonu)
y_train = Auto_train['mpg']
# Inicijalizacija OLS (Ordinary Least Squares) modela
model = sm.OLS(y_train, X_train)
# Treniranje (fitovanje) modela na trening podacima
results = model.fit()

# Evaluacija modela na validacionom skupu
# Transformacija validacionih podataka koristeći istu specifikaciju
X_valid = hp_mm.transform(Auto_valid)
# Definisanje zavisne varijable za validaciju
y_valid = Auto_valid['mpg']
# Generisanje predikcija za validacioni skup koristeći trenirani model
valid_pred = results.predict(X_valid)

# Izračunavanje i ispisivanje srednje kvadratne greške (MSE)
# Računanje proseka kvadrata razlika između stvarnih i predviđenih vrednosti
mse = np.mean((y_valid - valid_pred)**2)
# Ispisivanje rezultata MSE
print("MSE (Validacioni set - linearni):", mse)

## 1.2 Polinomska regresija sa validacijom
Potrebno je adaptirati imena kolona u eval_mse funkciji kao i imena varijabli koje se prosleđuju ovoj funkciji. 

In [ ]:
# Funkcija za izračunavanje MSE za zadati stepen polinoma
def eval_mse(degree, train, valid):
    # Kreiranje specifikacije modela sa polinomskim članom zadatog stepena
    mm = MS([poly('horsepower', degree=degree)]) 
    # Transformacija trening podataka
    X_train = mm.fit_transform(train)
    # Transformacija validacionih podataka
    X_valid = mm.transform(valid)
    # Izdvajanje zavisne varijable za trening
    y_train = train['mpg']
    # Izdvajanje zavisne varijable za validaciju
    y_valid = valid['mpg']
    
    # Treniranje OLS modela na trening podacima
    results = sm.OLS(y_train, X_train).fit()
    # Predviđanje vrednosti na validacionom skupu
    valid_pred = results.predict(X_valid)
    
    # Vraćanje izračunate srednje kvadratne greške (MSE)
    return np.mean((y_valid - valid_pred)**2)

# Izračunavanje MSE za polinome stepena 1, 2 i 3
mse_degree_1 = eval_mse(1, Auto_train, Auto_valid) # Linearni model
mse_degree_2 = eval_mse(2, Auto_train, Auto_valid) # Kvadratni model
mse_degree_3 = eval_mse(3, Auto_train, Auto_valid) # Kubni model

# Ispisivanje MSE vrednosti za poređenje
print("MSE za stepene 1, 2, 3:", mse_degree_1, mse_degree_2, mse_degree_3)

## 1.3 Unakrsna validacija (Cross-Validation) 
Potrebno je adaptirati imena kolona koje se koriste za treniranje lineranog modela.

Zavisna varijabla (y_train, y_valid) treba da bude Rings kolona iz abalone_dataset.csv

Nezavisna varijabla treba da bude Shell weight kolona iz abalone_dataset.csv

In [ ]:
# Priprema podataka za sklearn (koristeći ceo skup podataka jer CV radi podelu automatski)
hp_model = MS(['horsepower'])
# Transformacija svih prediktora
X = hp_model.fit_transform(Auto)
# Izdvajanje zavisne varijable iz celog skupa
y = Auto['mpg']
# Inicijalizacija modela linearne regresije iz scikit-learn biblioteke
model = LinearRegression()

# Unakrsna validacija "izostavi-jedan" (Leave-One-Out Cross-Validation - LOOCV)
# Koristimo KFold gde je broj podela (splits) jednak ukupnom broju opservacija
cv = KFold(n_splits=len(X), shuffle=True, random_state=0)
# Izvođenje unakrsne validacije; scoring='neg_mean_squared_error' vraća negativni MSE
cv_results = cross_validate(model, X, y, cv=cv, scoring='neg_mean_squared_error')
# Računanje prosečnog MSE (menjamo znak jer sklearn vraća negativne vrednosti)
loocv_mse = -np.mean(cv_results['test_score'])
# Ispisivanje rezultata LOOCV
print('LOOCV MSE:', loocv_mse)

# k-Fold unakrsna validacija (npr. k=10)
# Definisanje 10-struke podele (10-fold splitter)
cv_10 = KFold(n_splits=10, shuffle=True, random_state=0)
# Izvođenje 10-struke unakrsne validacije
cv_results_10 = cross_validate(model, X, y, cv=cv_10, scoring='neg_mean_squared_error')
# Računanje prosečnog MSE za 10-struku validaciju
kfold_mse = -np.mean(cv_results_10['test_score'])
# Ispisivanje rezultata 10-Fold CV
print('10-Fold CV MSE:', kfold_mse)

## Zadatak 2 - Selekcija nezavisnih prediktora (Forward selection)
U ovom zadatku vršimo selekciju nezavisnih prediktora na abalone_dataset.csv.

Potrebno je adaptirati kod da zavisna varijabla "Y" treba da bude "Rings" kolona iz abalone_dataset.csv dok sve preostale varijable iz abalone_dataset.csv (sem "Sex" kolone) trebaju da budu nezavisne varijable.

Za kriterijum selekcije prediktora prvo koristimo scoring="neg_mean_squared_error".

Zatim koristimo Mallows Cp kriterijum definisan u funkciji nCP.

Mallows Cp je kriterijum za izbor modela u linearnoj regresiji koji se koristi za biranje između konkurentskih modela tako što uravnotežuje kvalitet prilagođavanja (dobrotu uklapanja) i složenost modela. Njegova osnovna svrha je procena očekivane greške na test skupu bez eksplicitne upotrebe unakrsne validacije.




In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from matplotlib.pyplot import subplots
from statsmodels.api import OLS
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP.models import (Stepwise, sklearn_selected, sklearn_selection_path)
from sklearn.preprocessing import StandardScaler


In [ ]:
from l0bnb import fit_path

In [ ]:
Hitters = load_data('Hitters') # Učitavanje "Hitters" skupa podataka
np.isnan(Hitters['Salary']).sum() # Računa ukupan broj nedostajućih vrednosti u koloni 'Salary'
Hitters = Hitters.dropna() # Uklanja sve redove koji sadrže nedostajuće vrednosti
Hitters.shape # Prikazuje dimenzije pročišćenog skupa podataka

In [ ]:
def nCp(sigma2, estimator, X, Y):
    "Negative Cp statistic"
    n, p = X.shape # Dobija broj uzoraka (n) i broj prediktora (p)
    Yhat = estimator.predict(X) # Generiše predviđanja na osnovu estimatora
    RSS = np.sum((Y - Yhat)**2) # Računa sumu kvadrata reziduala (RSS)
    return -(RSS + 2 * p * sigma2) / n # Vraća negativnu Cp statistiku (jer sklearn maksimizuje skor)

In [ ]:
design = MS(Hitters.columns.drop('Salary')).fit(Hitters) # Kreira dizajn matricu isključujući ciljnu promenljivu 'Salary'
Y = np.array(Hitters['Salary']) # Izdvaja ciljnu promenljivu Y kao niz
X = design.transform(Hitters) # Transformiše podatke u matricu prediktora X
sigma2 = OLS(Y,X).fit().scale # Prilagođava OLS model na svim podacima da bi se procenila varijansa šuma (sigma2)

In [ ]:
neg_Cp = partial(nCp, sigma2) # Kreira novu funkciju skorovanja sa fiksiranim parametrom sigma2 (procenjena varijansa šuma)

In [ ]:
strategy = Stepwise.first_peak(design, direction='forward', max_terms=len(design.terms)) # Definiše strategiju postupnog izbora (forward selection) koja staje na prvom piku

In [ ]:
hitters_MSE = sklearn_selected(OLS, strategy, scoring="neg_mean_squared_error") # Inicijalizuje selektor modela koristeći OLS i definisanu strategiju (podrazumevano koristi MSE)
hitters_MSE.fit(Hitters, Y) # Pokreće proces selekcije na podacima
hitters_MSE.selected_state_ # Prikazuje izabrane promenljive (trebalo bi da budu sve ako se koristi MSE)

In [ ]:
hitters_Cp = sklearn_selected(OLS, strategy, scoring=neg_Cp) # Inicijalizuje selektor modela koristeći negativni Cp kao kriterijum
hitters_Cp.fit(Hitters, Y) # Pokreće proces selekcije
hitters_Cp.selected_state_ # Prikazuje izabrane promenljive prema Cp kriterijumu

## Zadatak 3 - Ridge i Lasso regresija - optimizacija lambda parametra sa unakrsnom validacijom

Potrebno je adaptirati kod da zavisna varijabla "Y" treba da bude "Rings" kolona iz abalone_dataset.csv dok sve preostale varijable iz abalone_dataset.csv (sem "Sex" kolone) trebaju da budu nezavisne varijable.

In [ ]:
D = design.fit_transform(Hitters) # Priprema dizajn matricu
X = np.asarray(D) 
scaler = StandardScaler(with_mean=True, with_std=True) # Skaliranje ulaynih podataka
X_s = scaler.fit_transform(X)
lambdas = 10**np.linspace(8, -2, 100) / Y.std() # Generiše niz lambda vrednosti (hiperparametara) skaliranih devijacijom Y
soln_array = skl.ElasticNet.path(X_s, Y, l1_ratio=0, alphas=lambdas)[1] # Računa putanju rešenja za Ridge regresiju (l1_ratio=0)
soln_array.shape # Prikazuje dimenzije rešenja

In [ ]:
soln_path = pd.DataFrame(soln_array.T, columns=D.columns, index=-np.log(lambdas)) # Kreira DataFrame sa koeficijentima za lakše plotovanje
soln_path.index.name = 'negative log(lambda)' # Imenuje indeks DataFrame-a
soln_path # Prikazuje DataFrame

In [ ]:
path_fig, ax = subplots(figsize=(8,8)) # Kreira figuru za grafik
soln_path.plot(ax=ax, legend=False) # Iscrtava promene koeficijenata u zavisnosti od lambde
ax.set_xlabel('$-\log(\lambda)$', fontsize=20) # Postavlja oznaku x-ose (koristi LaTeX formatiranje)
ax.set_ylabel('Standardized coefficients', fontsize=20) # Postavlja oznaku y-ose
ax.legend(loc='upper left') # Postavlja legendu u gornji levi ugao

In [ ]:
beta_hat = soln_path.loc[soln_path.index[39]] # Izdvaja koeficijente za 40-ti korak
lambdas[39], beta_hat # Prikazuje lambda vrednost i koeficijente tog koraka

In [ ]:
np.linalg.norm(beta_hat) # Računa L2 normu koeficijenata za izabrani model

In [ ]:
beta_hat = soln_path.loc[soln_path.index[59]] # Izdvaja koeficijente za 60-ti korak (manja lambda)
lambdas[59], np.linalg.norm(beta_hat) # Prikazuje lambda vrednost i veću L2 normu

In [ ]:
ridge = skl.ElasticNet(alpha=lambdas[59], l1_ratio=0) # Inicijalizuje Ridge model sa specifičnom lambdom
scaler = StandardScaler(with_mean=True, with_std=True) # Inicijalizuje skaler za standardizaciju podataka
pipe = Pipeline(steps=[('scaler', scaler), ('ridge', ridge)]) # Kreira pajplajn koji prvo skalira pa primenjuje Ridge
pipe.fit(X, Y) # Trenira pajplajn na podacima

In [ ]:
np.linalg.norm(ridge.coef_) # Proverava L2 normu koeficijenata iz pajplajna (treba da se poklapa)

In [ ]:
validation = skm.ShuffleSplit(n_splits=1, test_size=0.5, random_state=0) # Kreira validacioni split (50/50)
ridge.alpha = 0.01 # Postavlja proizvoljnu vrednost za lambda (alpha)
results = skm.cross_validate(ridge, X, Y, scoring='neg_mean_squared_error', cv=validation) # Izvršava validaciju i računa negativni MSE
-results['test_score'] # Prikazuje pozitivan MSE

In [ ]:
ridge.alpha = 1e10 # Postavlja ogromnu vrednost za lambda (teži ka nultom modelu)
results = skm.cross_validate(ridge, X, Y, scoring='neg_mean_squared_error', cv=validation) # Izvršava validaciju
-results['test_score'] # Prikazuje MSE (koji bi trebao biti sličan varijansi Y)

In [ ]:
param_grid = {'ridge__alpha': lambdas} # Definiše mrežu parametara za pretragu (lambdas)
grid = skm.GridSearchCV(pipe, param_grid, cv=validation, scoring='neg_mean_squared_error') # Inicijalizuje pretragu na mreži (Grid Search) sa validacionim setom
grid.fit(X, Y) # Pokreće pretragu da nađe najbolju lambdu
grid.best_params_['ridge__alpha'] # Prikazuje najbolju pronađenu vrednost za lambda
grid.best_estimator_ # Prikazuje najbolji estimator (ceo pajplajn)

In [ ]:
K = 5
kfold = skm.KFold(K,
random_state=0,
shuffle=True)

grid = skm.GridSearchCV(pipe, param_grid, cv=kfold, scoring='neg_mean_squared_error') # Inicijalizuje Grid Search koristeći 5-fold unakrsnu validaciju
grid.fit(X, Y) # Pokreće pretragu
grid.best_params_['ridge__alpha'] # Prikazuje najbolju lambdu prema CV
grid.best_estimator_ # Prikazuje najbolji estimator

In [ ]:
ridge_fig, ax = subplots(figsize=(8,8)) # Kreira figuru
ax.errorbar(-np.log(lambdas), -grid.cv_results_['mean_test_score'], yerr=grid.cv_results_['std_test_score'] / np.sqrt(K)) # Iscrtava CV greške sa standardnim greškama
ax.set_ylim([50000, 250000]) # Postavlja limite y-ose
ax.set_xlabel('$-\log(\lambda)$', fontsize=20) # Postavlja oznaku x-ose
ax.set_ylabel('Cross-validated MSE', fontsize=20) # Postavlja oznaku y-ose

In [ ]:
grid_r2 = skm.GridSearchCV(pipe, param_grid, cv=kfold) # Inicijalizuje Grid Search koristeći podrazumevani R2 skor
grid_r2.fit(X, Y) # Pokreće pretragu

In [ ]:
r2_fig, ax = subplots(figsize=(8,8)) # Kreira figuru za R2 grafik
ax.errorbar(-np.log(lambdas), grid_r2.cv_results_['mean_test_score'], yerr=grid_r2.cv_results_['std_test_score'] / np.sqrt(K)) # Iscrtava R2 skorove
ax.set_xlabel('$-\log(\lambda)$', fontsize=20) # Oznaka x-ose
ax.set_ylabel('Cross-validated $R^2$', fontsize=20) # Oznaka y-ose

In [ ]:
ridgeCV = skl.ElasticNetCV(alphas=lambdas, l1_ratio=0, cv=kfold) # Koristi efikasniju ugrađenu CV funkciju za Ridge
pipeCV = Pipeline(steps=[('scaler', scaler), ('ridge', ridgeCV)]) # Pravi pajplajn sa RidgeCV
pipeCV.fit(X, Y) # Trenira pajplajn

In [ ]:
tuned_ridge = pipeCV.named_steps['ridge'] # Izdvaja istrenirani Ridge model iz pajplajna
ridgeCV_fig, ax = subplots(figsize=(8,8)) # Kreira figuru
ax.errorbar(-np.log(lambdas), tuned_ridge.mse_path_.mean(1), yerr=tuned_ridge.mse_path_.std(1) / np.sqrt(K)) # Iscrtava putanju MSE grešaka
ax.axvline(-np.log(tuned_ridge.alpha_), c='k', ls='--') # Označava vertikalnom linijom izabranu optimalnu lambdu
ax.set_ylim([50000, 250000]) # Limiti y-ose
ax.set_xlabel('$-\log(\lambda)$', fontsize=20) # Oznaka x-ose
ax.set_ylabel('Cross-validated MSE', fontsize=20) # Oznaka y-ose

In [ ]:
np.min(tuned_ridge.mse_path_.mean(1)) # Prikazuje minimalnu srednju MSE vrednost sa putanje

In [ ]:
tuned_ridge.coef_ # Prikazuje koeficijente modela za optimalnu lambdu (svi su različiti od nule kod Ridge-a)

In [ ]:
outer_valid = skm.ShuffleSplit(n_splits=1, test_size=0.25, random_state=1) # Definisanje spoljašnjeg validacionog splita
inner_cv = skm.KFold(n_splits=5, shuffle=True, random_state=2) # Definisanje unutrašnje CV petlje
ridgeCV = skl.ElasticNetCV(alphas=lambdas, l1_ratio=0, cv=inner_cv) # Definisanje modela sa unutrašnjom CV
pipeCV = Pipeline(steps=[('scaler', scaler), ('ridge', ridgeCV)]) # Kreiranje pajplajna
results = skm.cross_validate(pipeCV, X, Y, cv=outer_valid, scoring='neg_mean_squared_error') # Izvršavanje ugnježdene validacije
-results['test_score'] # Prikaz rezultujućeg MSE-a

In [ ]:
lassoCV = skl.ElasticNetCV(n_alphas=100, l1_ratio=1, cv=kfold) # Inicijalizuje CV za Lasso (l1_ratio=1)
pipeCV = Pipeline(steps=[('scaler', scaler), ('lasso', lassoCV)]) # Kreira pajplajn za Lasso
pipeCV.fit(X, Y) # Trenira Lasso model
tuned_lasso = pipeCV.named_steps['lasso'] # Izdvaja Lasso model
tuned_lasso.alpha_ # Prikazuje optimalnu lambdu za Lasso

In [ ]:
lambdas, soln_array = skl.Lasso.path(X_s, Y, l1_ratio=1, n_alphas=100)[:2] # Računa putanju rešenja za Lasso
soln_path = pd.DataFrame(soln_array.T, columns=D.columns, index=-np.log(lambdas)) # Kreira DataFrame koeficijenata

In [ ]:
path_fig, ax = subplots(figsize=(8,8)) # Kreira figuru
soln_path.plot(ax=ax, legend=False) # Iscrtava Lasso putanju
ax.legend(loc='upper left') # Legenda
ax.set_xlabel('$-\log(\lambda)$', fontsize=20) # Oznaka x-ose
ax.set_ylabel('Standardized coefficients', fontsize=20) # Oznaka y-ose

In [ ]:
np.min(tuned_lasso.mse_path_.mean(1)) # Prikazuje minimalni MSE za Lasso

In [ ]:
lassoCV_fig, ax = subplots(figsize=(8,8)) # Kreira figuru za CV grešku Lasso-a
ax.errorbar(-np.log(tuned_lasso.alphas_), tuned_lasso.mse_path_.mean(1), yerr=tuned_lasso.mse_path_.std(1) / np.sqrt(K)) # Iscrtava greške
ax.axvline(-np.log(tuned_lasso.alpha_), c='k', ls='--') # Označava optimalnu lambdu
ax.set_ylim([50000, 250000]) # Limiti y-ose
ax.set_xlabel('$-\log(\lambda)$', fontsize=20) # Oznaka x-ose
ax.set_ylabel('Cross-validated MSE', fontsize=20) # Oznaka y-ose

In [ ]:
tuned_lasso.coef_ # Prikazuje koeficijente Lasso modela (mnogi su nula - sparse model)